In [ ]:
%use kandy
%use dataframe

aIn the notebook **Kandy** is used to plot graphs and **Dataframe** to manipulate csv files.

The following are the classes used to define a Game:

In [ ]:
data class Game(
    val numberOfVictories: Int,
    val numberOfLosses: Int,
) {
    init {
        if (numberOfVictories !in 0..10)
            throw IllegalArgumentException("The number of victories must be between 0 and 10")
    }
    val flawless: Boolean = numberOfVictories == 10 && numberOfLosses == 0
}


In [ ]:
enum class Victory {
    FLAWLESS,
    GOLD,
    SILVER,
    BRONZE,
    LOSS
}

fun formatVictory(game: Game): Victory {
    return if (game.flawless) {
        Victory.FLAWLESS
    }
    else {
        when (game.numberOfVictories) {
            10 -> Victory.GOLD
            in 7..9 -> Victory.SILVER
            in 4..6 -> Victory.BRONZE
            else -> Victory.LOSS
        }
    }
}

In [ ]:
enum class Hero {
    VANESSA,
    DOOLEY,
    PYGMALIEN,
    MAK,
    STELLE
}

fun formatHero(hero: Hero): String {
    return when (hero) {
        Hero.VANESSA -> "Vanessa"
        Hero.DOOLEY -> "Dooley"
        Hero.PYGMALIEN -> "Pygmalien"
        Hero.MAK -> "Mak"
        Hero.STELLE -> "Stelle"
    }
}

A simple input loop to enter one or more games at a time:

In [ ]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import org.jetbrains.kotlinx.dataframe.io.writeCSV

data class GameRow(
    val Hero: String,
    val Victories: Int,
    val Losses: Int,
    val Category: String
)

val file = File("./../data/games.csv")

val existingGames: MutableList<GameRow> = if (file.exists()) {
    file.readLines().drop(1) //skip header
        .map { line ->
            val parts = line.split(",")
            GameRow(
                Hero = parts[0],
                Victories = parts[1].toInt(),
                Losses = parts[2].toInt(),
                Category = parts[3]
            )
        }.toMutableList()
} else {
    mutableListOf()
}

while (true) {
    print("Enter hero (VANESSA, DOOLEY, PYGMALIEN, MAK, STELLE) or 'quit': ")
    val heroInput = readLine()?.trim() ?: ""
    if (heroInput.equals("quit", ignoreCase = true)) break

    val hero = try {
        Hero.valueOf(heroInput.uppercase())
    } catch (e: Exception) {
        println("Invalid hero, defaulting to VANESSA")
        Hero.VANESSA
    }

    print("Victories (0–10): ")
    val victories = readLine()?.toIntOrNull() ?: 0

    print("Losses: ")
    val losses = readLine()?.toIntOrNull() ?: 0

    val game = Game(victories, losses)
    val category = formatVictory(game)

    existingGames.add(
        GameRow(
            Hero = formatHero(hero),
            Victories = game.numberOfVictories,
            Losses = game.numberOfLosses,
            Category = category.toString()
        )
    )
}

if (existingGames.isNotEmpty()) {
    val df = existingGames.toDataFrame()
    df.writeCsv(file)
    println("✅ Updated ${file.absolutePath} with ${existingGames.size} total games")
} else {
    println("No games entered.")
}


In [ ]:
val displayCsv = DataFrame.readCsv(file)
displayCsv